# Bond Profit Analysis (5-Year Horizon)

This notebook converts the provided script into a reproducible analysis with charts.

> Not financial advice. Results depend entirely on the assumptions and inputs below.


In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.max_rows = 200
pd.options.display.max_columns = 200


In [2]:
# Bonds transcribed from your screenshot (edit/add rows as needed)
bonds = [
    {"id":"META 5.625 2055-11-15", "issuer":"Meta", "coupon_pct":5.625, "ytm_pct":5.95, "ttm_years":29 + 285/365},
    {"id":"ORCL 5.95 2055-09-26", "issuer":"Oracle", "coupon_pct":5.95, "ytm_pct":6.62, "ttm_years":29 + 234/365},
    {"id":"SWMA 4.0 2028-05-31", "issuer":"Swedish Match", "coupon_pct":4.0, "ytm_pct":4.30, "ttm_years":2 + 117/365},
    {"id":"ORCL 6.0 2055-08-03", "issuer":"Oracle", "coupon_pct":6.0, "ytm_pct":6.68, "ttm_years":29 + 180/365},
    {"id":"ORCL 5.875 2045-09-26", "issuer":"Oracle", "coupon_pct":5.875, "ytm_pct":6.53, "ttm_years":19 + 234/365},
    {"id":"MRK 5.55 2055-12-04", "issuer":"Merck", "coupon_pct":5.55, "ytm_pct":5.63, "ttm_years":29 + 304/365},
    {"id":"JEF 6.2 2034-04-14", "issuer":"Jefferies", "coupon_pct":6.2, "ytm_pct":5.39, "ttm_years":8 + 69/365},
    {"id":"XAI 12.5 2030-06-30", "issuer":"xAI", "coupon_pct":12.5, "ytm_pct":8.76, "ttm_years":4 + 146/365},
    {"id":"META 5.5 2045-11-15", "issuer":"Meta", "coupon_pct":5.5, "ytm_pct":5.78, "ttm_years":19 + 285/365},
    {"id":"ORCL 5.2 2035-09-26", "issuer":"Oracle", "coupon_pct":5.2, "ytm_pct":5.61, "ttm_years":9 + 234/365},
    {"id":"ORCL 5.375 2054-09-27", "issuer":"Oracle", "coupon_pct":5.375, "ytm_pct":6.64, "ttm_years":28 + 235/365},
    {"id":"PAYX 5.35 2032-04-15", "issuer":"Paychex", "coupon_pct":5.35, "ytm_pct":4.79, "ttm_years":6 + 71/365},
    {"id":"APO 5.15 2035-08-12", "issuer":"Apollo", "coupon_pct":5.15, "ytm_pct":5.23, "ttm_years":9 + 189/365},
    {"id":"META 4.875 2035-11-15", "issuer":"Meta", "coupon_pct":4.875, "ytm_pct":5.02, "ttm_years":9 + 285/365},
    {"id":"ORCL 2.95 2030-04-01", "issuer":"Oracle", "coupon_pct":2.95, "ytm_pct":4.85, "ttm_years":4 + 56/365},
    {"id":"BOWL ~6 2031-01-23", "issuer":"Blue Owl", "coupon_pct":6.0, "ytm_pct":6.47, "ttm_years":4 + 354/365},
]


In [3]:
# -------- Assumptions (EDIT THESE) --------
HORIZON = 5.0
REINVEST = 0.04                 # reinvest rate for bonds that mature before 5y

sp_arith_mu = 0.08              # assumed S&P expected annual return
sp_vol = 0.18                   # assumed S&P annual volatility

sigma_y_annual = 0.0075         # assumed annual yield volatility for bonds you sell at year 5
# -----------------------------------------

FREQ = 2
FACE = 100.0
N = 80_000  # Monte Carlo paths (reduce for faster runs)


In [4]:
def bond_price(face, coupon_pct, ytm_pct, maturity_years, freq=2):
    if maturity_years <= 0:
        return face
    n = max(1, int(round(maturity_years * freq)))
    c = face * (coupon_pct / 100) / freq
    r = (ytm_pct / 100) / freq
    t = np.arange(1, n + 1)
    return float(np.sum(c / ((1 + r) ** t)) + face / ((1 + r) ** n))

def mod_duration(face, coupon_pct, ytm_pct, maturity_years, freq=2):
    if maturity_years <= 0:
        return 0.0
    n = max(1, int(round(maturity_years * freq)))
    c = face * (coupon_pct / 100) / freq
    r = (ytm_pct / 100) / freq
    t = np.arange(1, n + 1)
    cf = np.full(n, c, dtype=float)
    cf[-1] += face
    pv = cf / ((1 + r) ** t)
    price = pv.sum()
    macaulay = (t / freq * pv).sum() / price
    return float(macaulay / (1 + r))

def bond_hpr(b, horizon_years=5.0, reinvest_rate=0.04, delta_y=0.0):
    y0, cpn, T0 = b["ytm_pct"], b["coupon_pct"], b["ttm_years"]
    P0 = bond_price(FACE, cpn, y0, T0, FREQ)

    if T0 <= horizon_years:
        n_total = max(1, int(round(T0 * FREQ)))
        coupon_cash = (FACE * (cpn / 100) / FREQ) * n_total
        proceeds = coupon_cash + FACE
        rem = horizon_years - T0
        end_value = proceeds * ((1 + reinvest_rate) ** rem)
        return end_value / P0 - 1.0
    else:
        n_hold = max(1, int(round(horizon_years * FREQ)))
        coupon_cash = (FACE * (cpn / 100) / FREQ) * n_hold
        y1 = max(0.0, y0 + delta_y)
        P1 = bond_price(FACE, cpn, y1, T0 - horizon_years, FREQ)
        return (coupon_cash + P1) / P0 - 1.0


In [5]:
# Monte Carlo for S&P and bonds
rng = np.random.default_rng(42)

mu_log = math.log(1 + sp_arith_mu) - 0.5 * sp_vol**2
z = rng.standard_normal(N)
sp_gross = np.exp(mu_log * HORIZON + sp_vol * math.sqrt(HORIZON) * z)
sp_ret = sp_gross - 1.0

sigma_y_5y = sigma_y_annual * math.sqrt(HORIZON)

rows = []
ret_by_bond = {}

for b in bonds:
    P0 = bond_price(FACE, b["coupon_pct"], b["ytm_pct"], b["ttm_years"], FREQ)
    dur = mod_duration(FACE, b["coupon_pct"], b["ytm_pct"], b["ttm_years"], FREQ)

    if b["ttm_years"] <= HORIZON:
        bond_ret = np.full(N, bond_hpr(b, HORIZON, REINVEST, 0.0))
    else:
        dy = rng.normal(0.0, sigma_y_5y, size=N) * 100  # percentage points
        bond_ret = np.array([bond_hpr(b, HORIZON, REINVEST, d) for d in dy])

    ret_by_bond[b["id"]] = bond_ret

    bond_ann = (1.0 + bond_ret) ** (1 / HORIZON) - 1.0
    p10 = float(np.percentile(bond_ann, 10))
    p90 = float(np.percentile(bond_ann, 90))
    p_win = float(np.mean(bond_ret > sp_ret))

    rows.append({
        "bond": b["id"],
        "issuer": b["issuer"],
        "coupon%": b["coupon_pct"],
        "ytm%": b["ytm_pct"],
        "ttm_years": b["ttm_years"],
        "price_per_100": P0,
        "mod_dur_yrs": dur,
        "bond_ann_median%": float(np.median(bond_ann)) * 100,
        "bond_ann_10%": p10 * 100,
        "bond_ann_90%": p90 * 100,
        "P(bond>S&P)": p_win,
    })

df = pd.DataFrame(rows)
df.head()


,bond,issuer,coupon%,ytm%,ttm_years,price_per_100,mod_dur_yrs,bond_ann_median%,bond_ann_10%,bond_ann_90%,P(bond>S&P)
0,META 5.625 2055-11-15,Meta,5.625,5.95,29.780822,95.478531,14.062181,5.371231,1.275777,10.541856,0.473813
1,ORCL 5.95 2055-09-26,Oracle,5.950,6.62,29.641096,91.361049,13.147979,5.871694,2.137600,10.634141,0.496250
2,SWMA 4.0 2028-05-31,Swedish Match,4.000,4.30,2.320548,99.296049,2.352877,4.236578,4.236578,4.236578,0.406150
3,ORCL 6.0 2055-08-03,Oracle,6.000,6.68,29.493151,91.285548,13.067866,5.955116,2.207906,10.655705,0.498287
4,ORCL 5.875 2045-09-26,Oracle,5.875,6.53,19.641096,92.834574,11.184757,5.837951,2.774945,9.380174,0.486725


In [6]:
# Summary table
df_table = df.copy()
df_table["ttm_years"] = df_table["ttm_years"].round(2)
df_table["price_per_100"] = df_table["price_per_100"].round(2)
df_table["mod_dur_yrs"] = df_table["mod_dur_yrs"].round(1)
df_table["bond_ann_median%"] = df_table["bond_ann_median%"].round(2)
df_table["bond_ann_10_90%"] = df_table.apply(
    lambda r: f"{r['bond_ann_10%']:.2f}%–{r['bond_ann_90%']:.2f}%", axis=1
)
df_table["P(bond>S&P)"] = df_table["P(bond>S&P)"].round(3)

df_table = df_table.sort_values("P(bond>S&P)", ascending=False)
df_table[["bond", "issuer", "coupon%", "ytm%", "ttm_years", "price_per_100", "mod_dur_yrs",
         "bond_ann_median%", "bond_ann_10_90%", "P(bond>S&P)"]]


,bond,issuer,coupon%,ytm%,ttm_years,price_per_100,mod_dur_yrs,bond_ann_median%,bond_ann_10_90%,P(bond>S&P)
7,XAI 12.5 2030-06-30,xAI,12.500,8.76,4.40,113.67,3.5,7.07,7.07%–7.07%,0.541
3,ORCL 6.0 2055-08-03,Oracle,6.000,6.68,29.49,91.29,13.1,5.96,2.21%–10.66%,0.498
1,ORCL 5.95 2055-09-26,Oracle,5.950,6.62,29.64,91.36,13.1,5.87,2.14%–10.63%,0.496
10,ORCL 5.375 2054-09-27,Oracle,5.375,6.64,28.64,83.91,13.2,5.92,2.11%–10.69%,0.494
4,ORCL 5.875 2045-09-26,Oracle,5.875,6.53,19.64,92.83,11.2,5.84,2.77%–9.38%,0.487
15,BOWL ~6 2031-01-23,Blue Owl,6.000,6.47,4.97,98.02,4.2,5.83,5.83%–5.83%,0.482
0,META 5.625 2055-11-15,Meta,5.625,5.95,29.78,95.48,14.1,5.37,1.28%–10.54%,0.474
5,MRK 5.55 2055-12-04,Merck,5.550,5.63,29.83,98.85,14.4,5.10,0.90%–10.43%,0.462
8,META 5.5 2045-11-15,Meta,5.500,5.78,19.78,96.71,11.9,5.24,1.95%–9.06%,0.461
9,ORCL 5.2 2035-09-26,Oracle,5.200,5.61,9.64,97.01,7.4,5.10,3.72%–6.54%,0.448


## Screening / Filtering
Define your screen here, then run the cell to get a filtered list.


In [7]:
# --- Screening criteria (edit as desired) ---
min_p_win = 0.55            # minimum P(bond > S&P)
min_median_ann = 5.0        # minimum median annualized return (%)
max_duration = 15.0         # maximum modified duration (years)
min_ytm = None              # e.g., 5.0 to require ytm >= 5
max_price = None            # e.g., 110 to avoid large premiums
min_ttm = None              # e.g., 3.0 to avoid very short bonds
max_ttm = None              # e.g., 20.0 to avoid ultra-longs
issuer_allowlist = None     # e.g., ['xAI', 'Meta']
issuer_blocklist = None     # e.g., ['Oracle']
bond_name_contains = None   # e.g., 'xai' to match by name
# ------------------------------------------

screen = df_table.copy()
screen = screen[screen['P(bond>S&P)'] >= min_p_win]
screen = screen[screen['bond_ann_median%'] >= min_median_ann]
screen = screen[screen['mod_dur_yrs'] <= max_duration]

if min_ytm is not None:
    screen = screen[screen['ytm%'] >= min_ytm]
if max_price is not None:
    screen = screen[screen['price_per_100'] <= max_price]
if min_ttm is not None:
    screen = screen[screen['ttm_years'] >= min_ttm]
if max_ttm is not None:
    screen = screen[screen['ttm_years'] <= max_ttm]
if issuer_allowlist is not None:
    allow = {s.lower() for s in issuer_allowlist}
    screen = screen[screen['issuer'].str.lower().isin(allow)]
if issuer_blocklist is not None:
    block = {s.lower() for s in issuer_blocklist}
    screen = screen[~screen['issuer'].str.lower().isin(block)]
if bond_name_contains is not None:
    screen = screen[screen['bond'].str.lower().str.contains(bond_name_contains.lower())]

screen = screen.sort_values('P(bond>S&P)', ascending=False)
screen[["bond", "issuer", "bond_ann_median%", "bond_ann_10_90%", "P(bond>S&P)",
        "ytm%", "price_per_100", "mod_dur_yrs", "ttm_years"]]


,bond,issuer,bond_ann_median%,bond_ann_10_90%,P(bond>S&P),ytm%,price_per_100,mod_dur_yrs,ttm_years


In [8]:
# Top candidates by P(bond>S&P)
df_top = df_table.head(5)
df_top[["bond", "issuer", "bond_ann_median%", "bond_ann_10_90%", "P(bond>S&P)"]]


,bond,issuer,bond_ann_median%,bond_ann_10_90%,P(bond>S&P)
7,XAI 12.5 2030-06-30,xAI,7.07,7.07%–7.07%,0.541
3,ORCL 6.0 2055-08-03,Oracle,5.96,2.21%–10.66%,0.498
1,ORCL 5.95 2055-09-26,Oracle,5.87,2.14%–10.63%,0.496
10,ORCL 5.375 2054-09-27,Oracle,5.92,2.11%–10.69%,0.494
4,ORCL 5.875 2045-09-26,Oracle,5.84,2.77%–9.38%,0.487


In [9]:
# Chart 1: Probability bond beats S&P over the horizon
df_plot = df_table.sort_values("P(bond>S&P)", ascending=True)
fig = px.bar(
    df_plot,
    x="P(bond>S&P)",
    y="bond",
    orientation="h",
    color="issuer",
    title="Probability Bond Beats S&P (5-Year Horizon)",
)
fig.update_layout(xaxis_tickformat=".0%", yaxis_title="", xaxis_title="P(bond > S&P)")
fig.show()


In [10]:
# Chart 2: Median annualized return vs duration
fig = px.scatter(
    df,
    x="mod_dur_yrs",
    y="bond_ann_median%",
    color="issuer",
    size="ytm%",
    hover_data=["bond", "coupon%", "ytm%", "ttm_years", "price_per_100"],
    title="Median Bond Annualized Return vs Duration",
)
fig.update_layout(xaxis_title="Modified Duration (years)", yaxis_title="Median Annualized Return (%)")
fig.show()


In [11]:
# Optional: Distribution comparison for the top bond vs S&P
top_bond_id = df_table.iloc[0]["bond"]
top_bond_ret = ret_by_bond[top_bond_id]

fig = go.Figure()
fig.add_histogram(x=top_bond_ret, nbinsx=60, name=top_bond_id, opacity=0.6, histnorm="probability")
fig.add_histogram(x=sp_ret, nbinsx=60, name="S&P", opacity=0.6, histnorm="probability")
fig.update_layout(
    barmode="overlay",
    title="5-Year Total Return Distribution: Top Bond vs S&P",
    xaxis_title="Total Return",
    yaxis_title="Probability",
)
fig.show()
